In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
df=pd.read_csv('/content/drive/MyDrive/RomanUrduHateSpeech.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import string
import numpy as np
import pandas as pd
import joblib


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack

In [ ]:
df.isnull().sum()

,0
comment,3
label,16


In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

,0
comment,0
label,0


In [ ]:
df.shape

(170973, 2)

In [ ]:
def roman_urdu_normalization(text):
    """
    Handle common Roman Urdu spelling variations and abbreviations
    """
    if pd.isna(text):
        return ""

    text = str(text).lower().strip()

    # Common Roman Urdu variations mapping
    variations = {
    r'\bsee\b': 'si',        # was
    r'\bhey\b': 'hai',       # is
    r'\bh[ae]+\b': 'hai',     # is (to catch both 'hae' and 'he')
    r'\bhain\b': 'hain',     # are
    r'\btum\b': 'tu',        # you (informal)
    r'\bap\b': 'aap',        # you (formal)
    r'\bk\b': 'ke',          # of
    r'\bka\b': 'ka',         # of (male)
    r'\bki\b': 'ki',         # of (female)
    r'\bkoi\b': 'koi',       # someone
    r'\bkuch\b': 'kuch',     # something
    r'\bbs\b': 'bas',        # enough
    r'\bplz\b': 'please',    # please
    r'\bthx\b': 'thanks',    # thanks
    r'\bu\b': 'you',         # you
    r'\br\b': 'are',         # are
    r'\bur\b': 'aur',        # and
    r'\bjo\b': 'jo',         # that
    r'\bwo\b': 'wo',         # that
                  }


    for pattern, replacement in variations.items():
        text = re.sub(pattern, replacement, text)

    return text


In [ ]:
def enhanced_roman_urdu_cleaner(text):
    """
    Comprehensive cleaning pipeline for Roman Urdu text
    """
    if pd.isna(text) or text == "":
        return ""

    # Step 1: Convert to lowercase
    text = str(text).lower().strip()

    # Step 2: Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Step 3: Remove user mentions and hashtags (but keep the text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#(\w+)', r'\1', text)  # Remove # but keep the word

    # Step 4: Remove digits
    text = re.sub(r'\d+', '', text)

    # Step 5: Apply Roman Urdu normalization
    text = roman_urdu_normalization(text)

    # Step 6: Handle repeated characters (but be conservative)
    # Reduce 3+ repeated chars to 2: "looooove" -> "loove"
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Step 7: Selective punctuation handling
    # Remove most punctuation but keep emotional indicators
    text = re.sub(r'[#$%&*()+,\-/:;<=>@[\\]^_`{|}~]', ' ', text)

    # Step 8: Preserve emotional punctuation by converting to tokens
    text = re.sub(r'!+', ' ! ', text)      # Multiple ! -> single ! with spaces
    text = re.sub(r'\?+', ' ? ', text)     # Multiple ? -> single ? with spaces
    text = re.sub(r'\.{2,}', ' ... ', text) # Ellipsis

    # Step 9: Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
print("\n Applying enhanced Roman Urdu cleaning...")
df['cleaned_comment'] = df['comment'].apply(enhanced_roman_urdu_cleaner)


 Applying enhanced Roman Urdu cleaning...


In [ ]:
df.head()

,comment,label,cleaned_comment
0,hey watch my funny vines,False,hai watch my funny vines
1,ara mush watch our funny video on ram rahim au...,False,ara mush watch our funny video on ram rahim au...
2,kafi sehi vdo h yie,False,kafi sehi vdo h yie
3,hahhahahahha hahaha i just can t stop laughing...,False,hahhahahahha hahaha i just can t stop laughing...
4,hahaha i saw her videos too and i got cancer c...,False,hahaha i saw her videos too and i got cancer c...


In [ ]:
df=df.drop('comment',axis=1)

In [ ]:
df.head()

,label,cleaned_comment
0,False,hai watch my funny vines
1,False,ara mush watch our funny video on ram rahim au...
2,False,kafi sehi vdo h yie
3,False,hahhahahahha hahaha i just can t stop laughing...
4,False,hahaha i saw her videos too and i got cancer c...


In [ ]:
print("\n Filtering short comments...")
before_filter = len(df)

# Remove comments that are too short after cleaning
df = df[df['cleaned_comment'].str.len() >= 3]
df = df[df['cleaned_comment'].str.split().str.len() >= 2]

print(f"Removed {before_filter - len(df)} comments that were too short after cleaning")
print(f"Remaining dataset: {len(df)} rows")


 Filtering short comments...
Removed 1081 comments that were too short after cleaning
Remaining dataset: 169892 rows


In [ ]:

def add_roman_urdu_features(df, text_col='cleaned_comment'):
    """
    Add features specifically relevant for Roman Urdu abuse detection
    """

    # Basic text statistics
    df['word_count'] = df[text_col].apply(lambda x: len(str(x).split()))
    df['char_count'] = df[text_col].apply(len)
    df['avg_word_length'] = df[text_col].apply(
        lambda x: np.mean([len(word) for word in str(x).split()]) if len(str(x).split()) > 0 else 0
    )

    # Emotional and intensity indicators
    df['exclamation_count'] = df[text_col].apply(lambda x: str(x).count('!'))
    df['question_count'] = df[text_col].apply(lambda x: str(x).count('?'))
    df['ellipsis_count'] = df[text_col].apply(lambda x: str(x).count('...'))

    # Capitalization patterns (if any remain)
    df['uppercase_ratio'] = df[text_col].apply(
        lambda x: sum(1 for c in str(x) if c.isupper()) / len(str(x)) if len(str(x)) > 0 else 0
    )

    # Common abusive word indicators (you can expand this list)
    abusive_indicators = {
        'contains_kutti': r'\bkutti',
        'contains_harami': r'\bharami',
        'contains_bhen': r'\bbhen',
        'contains_chut': r'\bchut',
        'contains_laanat': r'\blaanat',
        'contains_gaali': r'\bgaali',
    }

    for feature_name, pattern in abusive_indicators.items():
        df[feature_name] = df[text_col].str.contains(pattern, case=False, na=False).astype(int)

    return df

print("\n Adding Roman Urdu specific features...")
df = add_roman_urdu_features(df, 'cleaned_comment')


 Adding Roman Urdu specific features...


In [ ]:
print("\n FINAL DATASET QUALITY CHECK:")
print(f"Final dataset shape: {df.shape}")
print(f"Label distribution:\n{df['label'].value_counts()}")
print(f"Missing values in final dataset:\n{df.isnull().sum()}")

print("\n Feature overview:")
print(f"Number of features: {len(df.columns)}")
print(f"Feature names: {df.columns.tolist()}")


 FINAL DATASET QUALITY CHECK:
Final dataset shape: (169892, 15)
Label distribution:
label
False    87526
True     82366
Name: count, dtype: int64
Missing values in final dataset:
label                0
cleaned_comment      0
word_count           0
char_count           0
avg_word_length      0
exclamation_count    0
question_count       0
ellipsis_count       0
uppercase_ratio      0
contains_kutti       0
contains_harami      0
contains_bhen        0
contains_chut        0
contains_laanat      0
contains_gaali       0
dtype: int64

 Feature overview:
Number of features: 15
Feature names: ['label', 'cleaned_comment', 'word_count', 'char_count', 'avg_word_length', 'exclamation_count', 'question_count', 'ellipsis_count', 'uppercase_ratio', 'contains_kutti', 'contains_harami', 'contains_bhen', 'contains_chut', 'contains_laanat', 'contains_gaali']


In [ ]:
df.isnull().sum()

,0
label,0
cleaned_comment,0
word_count,0
char_count,0
avg_word_length,0
exclamation_count,0
question_count,0
ellipsis_count,0
uppercase_ratio,0
contains_kutti,0


In [ ]:
df.head(20)

,label,cleaned_comment,word_count,char_count,avg_word_length,exclamation_count,question_count,ellipsis_count,uppercase_ratio,contains_kutti,contains_harami,contains_bhen,contains_chut,contains_laanat,contains_gaali
0,False,hai watch my funny vines,5,24,4.000000,0,0,0,0.0,0,0,0,0,0,0
1,False,ara mush watch our funny video on ram rahim au...,30,161,4.400000,0,0,0,0.0,0,0,0,0,0,0
2,False,kafi sehi vdo h yie,5,19,3.000000,0,0,0,0.0,0,0,0,0,0,0
3,False,hahhahahahha hahaha i just can t stop laughing...,24,128,4.375000,0,0,0,0.0,0,0,0,0,0,0
4,False,hahaha i saw her videos too and i got cancer c...,31,140,3.548387,0,0,0,0.0,0,0,0,0,0,0
5,False,awesome bro ek number banaya hai tune video,8,43,4.500000,0,0,0,0.0,0,0,0,0,0,0
6,False,rbg gamer watch our best funny video bihari v ...,32,171,4.375000,0,0,0,0.0,0,0,0,0,0,0
7,False,carryminati omg uhh replied,4,27,6.000000,0,0,0,0.0,0,0,0,0,0,0
8,False,galti bol diya bhai toh kaise hai aap sab ye hoga,11,49,3.545455,0,0,0,0.0,0,0,0,0,0,0
9,False,m subscriber congratz,3,21,6.333333,0,0,0,0.0,0,0,0,0,0,0


In [ ]:
print("\n FINAL DATASET STATISTICS:")
print(f"Total comments: {len(df)}")
print(f"Average words per comment: {df['word_count'].mean():.2f}")
print(f"Average characters per comment: {df['char_count'].mean():.2f}")
print(f"Comments with exclamation marks: {(df['exclamation_count'] > 0).sum()}")
print(f"Comments with question marks: {(df['question_count'] > 0).sum()}")

# Check abusive word indicators
abusive_features = [col for col in df.columns if col.startswith('contains_')]
for feature in abusive_features:
    count = df[feature].sum()
    print(f"Comments with {feature}: {count} ({count/len(df)*100:.1f}%)")


 FINAL DATASET STATISTICS:
Total comments: 169892
Average words per comment: 14.29
Average characters per comment: 71.96
Comments with exclamation marks: 258
Comments with question marks: 986
Comments with contains_kutti: 99 (0.1%)
Comments with contains_harami: 340 (0.2%)
Comments with contains_bhen: 3054 (1.8%)
Comments with contains_chut: 25019 (14.7%)
Comments with contains_laanat: 56 (0.0%)
Comments with contains_gaali: 673 (0.4%)


#Text Corpus Creation(Corpus is like dictionarty of text for our data)

In [ ]:

corpus_file_path = '/content/roman_urdu_corpus.txt'

# Write all cleaned comments to a text file (one per line)
with open(corpus_file_path, 'w', encoding='utf-8') as f:
    for comment in df['cleaned_comment']:
        if comment and len(comment.strip()) > 0:  # Skip empty comments
            f.write(comment.strip() + '\n')

print(f" Corpus prepared: {len(df)} comments saved to {corpus_file_path}")
print(f" Sample corpus entries:")
with open(corpus_file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 3:  # Show first 3 lines
            print(f"  {i+1}. {line.strip()}")
        else:
            break

 Corpus prepared: 169892 comments saved to /content/roman_urdu_corpus.txt
 Sample corpus entries:
  1. hai watch my funny vines
  2. ara mush watch our funny video on ram rahim aur achha lage to please subscribe sb ki masti aapka subscribe hammare liye gold coin ke barabar hai please suport us
  3. kafi sehi vdo h yie


In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import time

# Start timer
start_time = time.time()

# Train FastText model with optimized parameters for Roman Urdu
model = fasttext.train_unsupervised(
    input=corpus_file_path,
    model='skipgram',      # Better for semantic relationships
    dim=300,               # Standard dimension size
    ws=5,                  # Context window size
    minCount=5,            # Ignore very rare words
    minn=3,                # Min char n-gram (handles Roman Urdu variations)
    maxn=6,                # Max char n-gram
    neg=10,                # Negative samples
    wordNgrams=1,          # Use character n-grams
    loss='ns',             # Negative sampling (efficient)
    epoch=50,              # Training iterations
    lr=0.05,               # Learning rate
    thread=12
    verbose=2
    # Use multiple CPU cores
)

# Calculate training time
training_time = time.time() - start_time

print(f" FastText training completed in {training_time:.2f} seconds")
print(f" Vocabulary size: {len(model.words)} words")
print(f" Embedding dimension: {model.get_dimension()}")

# Save the trained model
model_path = '/content/roman_urdu_fasttext_model.bin'
model.save_model(model_path)
print(f" Model saved to: {model_path}")

 FastText training completed in 6210.52 seconds
 Vocabulary size: 21102 words
 Embedding dimension: 300
 Model saved to: /content/roman_urdu_fasttext_model.bin


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("\n🧪 STEP 3: Testing Embedding Quality...")

# Test words with common Roman Urdu variations
test_words = ['kuttiya', 'badtameez', 'acha', 'insan']

for word in test_words:
    try:
        similar_words = model.get_nearest_neighbors(word, k=5)
        print(f"\n Words similar to '{word}':")
        for score, similar_word in similar_words:
            print(f"   {similar_word}: {score:.3f}")
    except:
        print(f" '{word}' not in vocabulary")

# Test handling of misspelled words
print(f"\n Testing misspelling handling:")
misspelled_words = ['kutiya', 'kutya', 'badtamiz', 'accha']
for word in misspelled_words:
    try:
        vector = model.get_word_vector(word)
        similar = model.get_nearest_neighbors(word, k=3)
        print(f"'{word}' → Similar: {[w for _, w in similar]}")
    except:
        print(f"'{word}' → Not in vocabulary")


🧪 STEP 3: Testing Embedding Quality...
 'kuttiya' not in vocabulary
 'badtameez' not in vocabulary
 'acha' not in vocabulary
 'insan' not in vocabulary

 Testing misspelling handling:
'kutiya' → Not in vocabulary
'kutya' → Not in vocabulary
'badtamiz' → Not in vocabulary
'accha' → Not in vocabulary


In [ ]:
df.head()

,label,cleaned_comment,word_count,char_count,avg_word_length,exclamation_count,question_count,ellipsis_count,uppercase_ratio,contains_kutti,contains_harami,contains_bhen,contains_chut,contains_laanat,contains_gaali
0,False,hai watch my funny vines,5,24,4.000000,0,0,0,0.0,0,0,0,0,0,0
1,False,ara mush watch our funny video on ram rahim au...,30,161,4.400000,0,0,0,0.0,0,0,0,0,0,0
2,False,kafi sehi vdo h yie,5,19,3.000000,0,0,0,0.0,0,0,0,0,0,0
3,False,hahhahahahha hahaha i just can t stop laughing...,24,128,4.375000,0,0,0,0.0,0,0,0,0,0,0
4,False,hahaha i saw her videos too and i got cancer c...,31,140,3.548387,0,0,0,0.0,0,0,0,0,0,0


In [ ]:

print("\n🔄 STEP 4: Converting Comments to Sentence Vectors...")

import numpy as np
from sklearn.preprocessing import LabelEncoder
import fasttext  # ← ADD THIS

def comment_to_vector(comment, fasttext_model):
    """
    Convert a comment to a sentence vector by averaging word vectors
    """
    words = str(comment).split()
    word_vectors = []

    for word in words:
        try:
            vector = fasttext_model.get_word_vector(word)
            word_vectors.append(vector)
        except:
            continue  # Skip words not in vocabulary

    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.get_dimension())

    # Average all word vectors to get sentence vector
    return np.mean(word_vectors, axis=0)

print(" Loading FastText embeddings model...")
fasttext_model = fasttext.load_model('/content/roman_urdu_fasttext_model.bin')
print(f" FastText model loaded: {len(fasttext_model.words)} words")

# Convert all comments to sentence vectors USING FASTTEXT MODEL
print(" Converting comments to vectors...")
X_vectors = np.array([comment_to_vector(comment, fasttext_model) for comment in df['cleaned_comment']])  # ← USING fasttext_model

# PROPER LABEL ENCODING
print(" Encoding labels...")
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['label'])  # Converts True/False to 1/0

print(f" Conversion completed!")
print(f" Feature matrix shape: {X_vectors.shape}")
print(f" Labels shape: {y_encoded.shape}")
print(f" Label mapping: {label_encoder.classes_} → {label_encoder.transform(label_encoder.classes_)}")
print(f" Sample vector (first 10 values): {X_vectors[0][:10]}")
print(f" Sample labels (first 10): {y_encoded[:10]}")

# Save label encoder for future use
import joblib
label_encoder_path = '/content/label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_path)
print(f" Label encoder saved to: {label_encoder_path}")


🔄 STEP 4: Converting Comments to Sentence Vectors...
 Loading FastText embeddings model...
 FastText model loaded: 21102 words
 Converting comments to vectors...
 Encoding labels...
 Conversion completed!
 Feature matrix shape: (169892, 300)
 Labels shape: (169892,)
 Label mapping: [False True] → [0 1]
 Sample vector (first 10 values): [-0.13855813 -0.02161103  0.11057878 -0.10238332  0.16220503 -0.15518376
 -0.00597168  0.19641681  0.00780241  0.15629956]
 Sample labels (first 10): [0 0 0 0 0 0 0 0 0 0]
 Label encoder saved to: /content/label_encoder.pkl


In [ ]:
# ============================================================
# SAVE ESSENTIAL FILES TO GOOGLE DRIVE
# ============================================================

print("\n💾 SAVING ESSENTIAL FILES TO GOOGLE DRIVE...")

import joblib
import pandas as pd
import fasttext
import shutil
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create folder in Google Drive
drive_folder = '/content/drive/MyDrive/RomanUrdu_Abuse_Detection'
os.makedirs(drive_folder, exist_ok=True)
print(f"✅ Created folder: {drive_folder}")

# 1. Save Corpus File
print("\n🔧 Saving corpus file...")
corpus_drive_path = f'{drive_folder}/roman_urdu_corpus.txt'
shutil.copy('/content/roman_urdu_corpus.txt', corpus_drive_path)
print(f"✅ Corpus saved to: {corpus_drive_path}")

# 2. Save FastText Embeddings
print("\n🔧 Saving FastText embeddings...")
fasttext_drive_path = f'{drive_folder}/roman_urdu_fasttext_model.bin'
fasttext_model.save_model(fasttext_drive_path)  # Your loaded FastText model
print(f"✅ FastText embeddings saved to: {fasttext_drive_path}")

# 3. Save Label Encoder
print("\n🔧 Saving label encoder...")
label_encoder_drive_path = f'{drive_folder}/label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_drive_path)
print(f"✅ Label encoder saved to: {label_encoder_drive_path}")


# Verification
print("\n📁 VERIFICATION - All essential files saved:")
print("=" * 50)
essential_files = [
    corpus_drive_path,
    fasttext_drive_path,
    label_encoder_drive_path
]

for file_path in essential_files:
    if os.path.exists(file_path):
        print(f"✅ {os.path.basename(file_path)}")
    else:
        print(f"❌ {os.path.basename(file_path)}")

print(f"\n🎯 ALL ESSENTIAL FILES SAVED TO GOOGLE DRIVE!")
print(f"📁 Location: {drive_folder}")


💾 SAVING ESSENTIAL FILES TO GOOGLE DRIVE...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Created folder: /content/drive/MyDrive/RomanUrdu_Abuse_Detection

🔧 Saving corpus file...
✅ Corpus saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detection/roman_urdu_corpus.txt

🔧 Saving FastText embeddings...
✅ FastText embeddings saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detection/roman_urdu_fasttext_model.bin

🔧 Saving label encoder...
✅ Label encoder saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detection/label_encoder.pkl

📁 VERIFICATION - All essential files saved:
✅ roman_urdu_corpus.txt
✅ roman_urdu_fasttext_model.bin
✅ label_encoder.pkl

🎯 ALL ESSENTIAL FILES SAVED TO GOOGLE DRIVE!
📁 Location: /content/drive/MyDrive/RomanUrdu_Abuse_Detection


In [ ]:
print("\n STEP 5: Creating Train-Test Split...")

from sklearn.model_selection import train_test_split

# Split data: 80% training, 20% testing (using ENCODED labels)
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y_encoded,  # ← CHANGED: using y_encoded instead of y
    test_size=0.2,
    random_state=42,
    stratify=y_encoded  # ← CHANGED: using y_encoded
)

print(f" Data split completed!")
print(f" Training set: {X_train.shape[0]} samples")
print(f" Test set: {X_test.shape[0]} samples")
print(f" Training label distribution:")
print(f"   - Abusive (1): {np.sum(y_train == 1)} ({(np.sum(y_train == 1)/len(y_train))*100:.1f}%)")  # ← CHANGED: 1 instead of True
print(f"   - Non-Abusive (0): {np.sum(y_train == 0)} ({(np.sum(y_train == 0)/len(y_train))*100:.1f}%)")  # ← CHANGED: 0 instead of False


 STEP 5: Creating Train-Test Split...
 Data split completed!
 Training set: 135913 samples
 Test set: 33979 samples
 Training label distribution:
   - Abusive (1): 65893 (48.5%)
   - Non-Abusive (0): 70020 (51.5%)


In [ ]:
print("\n STEP 6: Training Classification Models...")

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import joblib

# Define models to test
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': LinearSVC(random_state=42, max_iter=1000),  # ← CHANGED TO LinearSVC
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"\n🔧 Training {name}...")

    # Train model (using ENCODED labels y_train)
    model.fit(X_train, y_train)  # ← y_train is now properly encoded (0/1)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Store results
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'f1_score': f1,
        'predictions': y_pred
    }

    print(f" {name} trained!")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   F1-Score: {f1:.4f}")

# Find best model
best_model_name = max(results, key=lambda x: results[x]['f1_score'])
best_model = results[best_model_name]['model']
print(f"\n BEST MODEL: {best_model_name}")
print(f" Best F1-Score: {results[best_model_name]['f1_score']:.4f}")


 STEP 6: Training Classification Models...

🔧 Training Logistic Regression...
 Logistic Regression trained!
   Accuracy: 0.8584
   F1-Score: 0.8584

🔧 Training SVM...
 SVM trained!
   Accuracy: 0.8589
   F1-Score: 0.8589

🔧 Training Random Forest...
 Random Forest trained!
   Accuracy: 0.8432
   F1-Score: 0.8432

 BEST MODEL: SVM
 Best F1-Score: 0.8589


In [ ]:
# ============================================================
# STEP 7: Save ALL Models & Results
# ============================================================

print("\n💾 STEP 7: Saving ALL Models & Results...")

# Save ALL three models
model_paths = {}
for name in models.keys():
    model_path = f'/content/drive/MyDrive/RomanUrdu_Abuse_Detection{name.lower().replace(" ", "_")}_model.pkl'
    joblib.dump(results[name]['model'], model_path)
    model_paths[name] = model_path
    print(f"✅ {name} saved to: {model_path}")

# # Save FastText model (already saved in Step 2)
# print(f"✅ FastText embeddings saved to: {model_path}")

# # Print final results comparison
# print("\n📊 FINAL MODEL COMPARISON:")
# print("=" * 50)
# for name, result in results.items():
#     print(f"{name:20} | Accuracy: {result['accuracy']:.4f} | F1-Score: {result['f1_score']:.4f}")

# print(f"\n🏆 BEST MODEL: {best_model_name}")
# print(f"🎯 All models saved and ready for testing!")

# ============================================================
# STEP 8: Test ALL Models with Sample Predictions
# ============================================================

# print("\n🧪 STEP 8: Testing ALL Models with Sample Predictions...")

# # Test all models on the same 5 test samples
# test_samples = X_test[:5]
# true_labels = y_test[:5]

# print(f"Testing on {len(test_samples)} samples...")
# print("=" * 60)

# for name in models.keys():
#     print(f"\n🔍 {name} Predictions:")
#     model = results[name]['model']
#     predictions = model.predict(test_samples)
#     probabilities = model.predict_proba(test_samples) if hasattr(model, 'predict_proba') else None

#     for i, (true, pred) in enumerate(zip(true_labels, predictions)):
#         status = "✅ CORRECT" if true == pred else "❌ WRONG"
#         prob_text = f" [Prob: {probabilities[i][1]:.3f}]" if probabilities is not None else ""
#         print(f"  Sample {i+1}: True={true}, Pred={pred} {status}{prob_text}")

# print(f"\n🎯 All models tested! Ready for real-time predictions!")


💾 STEP 7: Saving ALL Models & Results...
✅ Logistic Regression saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detectionlogistic_regression_model.pkl
✅ SVM saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detectionsvm_model.pkl
✅ Random Forest saved to: /content/drive/MyDrive/RomanUrdu_Abuse_Detectionrandom_forest_model.pkl


In [ ]:


print("\n🧪 STEP 8: Testing ALL Models with Sample Predictions...")

# Test all models on the same 5 test samples
test_samples = X_test[:5]
true_labels = y_test[:5]

print(f"Testing on {len(test_samples)} samples...")
print("=" * 60)

for name in models.keys():
    print(f"\n🔍 {name} Predictions:")
    model = results[name]['model']
    predictions = model.predict(test_samples)
    probabilities = model.predict_proba(test_samples) if hasattr(model, 'predict_proba') else None

    for i, (true, pred) in enumerate(zip(true_labels, predictions)):
        status = "✅ CORRECT" if true == pred else "❌ WRONG"
        prob_text = f" [Prob: {probabilities[i][1]:.3f}]" if probabilities is not None else ""
        print(f"  Sample {i+1}: True={true}, Pred={pred} {status}{prob_text}")

print(f"\n🎯 All models tested! Ready for real-time predictions!")


🧪 STEP 8: Testing ALL Models with Sample Predictions...
Testing on 5 samples...

🔍 Logistic Regression Predictions:
  Sample 1: True=0, Pred=0 ✅ CORRECT [Prob: 0.225]
  Sample 2: True=0, Pred=0 ✅ CORRECT [Prob: 0.344]
  Sample 3: True=0, Pred=0 ✅ CORRECT [Prob: 0.029]
  Sample 4: True=1, Pred=1 ✅ CORRECT [Prob: 0.652]
  Sample 5: True=1, Pred=1 ✅ CORRECT [Prob: 0.922]

🔍 SVM Predictions:
  Sample 1: True=0, Pred=0 ✅ CORRECT
  Sample 2: True=0, Pred=0 ✅ CORRECT
  Sample 3: True=0, Pred=0 ✅ CORRECT
  Sample 4: True=1, Pred=1 ✅ CORRECT
  Sample 5: True=1, Pred=1 ✅ CORRECT

🔍 Random Forest Predictions:
  Sample 1: True=0, Pred=1 ❌ WRONG [Prob: 0.580]
  Sample 2: True=0, Pred=1 ❌ WRONG [Prob: 0.530]
  Sample 3: True=0, Pred=0 ✅ CORRECT [Prob: 0.050]
  Sample 4: True=1, Pred=1 ✅ CORRECT [Prob: 0.910]
  Sample 5: True=1, Pred=1 ✅ CORRECT [Prob: 0.920]

🎯 All models tested! Ready for real-time predictions!


In [ ]:
# ============================================================
# COMPLETE TESTING PIPELINE FOR ROMAN URDU ABUSE DETECTION
# ============================================================

print("🧪 TESTING SAVED MODELS WITH REAL COMMENTS...")

import fasttext
import joblib
import numpy as np
import pandas as pd
import re

# ============================================================
# STEP 1: LOAD ALL SAVED MODELS
# ============================================================

print("📥 STEP 1: Loading all saved models...")

# Load from Google Drive
drive_folder = '/content/drive/MyDrive/RomanUrdu_Abuse_Detection'

# 1. Load FastText model
fasttext_model = fasttext.load_model(f'{drive_folder}/roman_urdu_fasttext_model.bin')
print("✅ FastText model loaded")

# 2. Load Label Encoder
label_encoder = joblib.load(f'{drive_folder}/label_encoder.pkl')
print("✅ Label encoder loaded")

# 3. Load all classification models
models = {
    'Logistic Regression': joblib.load('/content/drive/MyDrive/RomanUrdu_Abuse_Detection/logistic_regression_model.pkl'),
    'SVM': joblib.load('/content/drive/MyDrive/RomanUrdu_Abuse_Detection/svm_model.pkl'),
    'Random Forest': joblib.load(f'/content/drive/MyDrive/RomanUrdu_Abuse_Detection/random_forest_model.pkl')
}
print("✅ All classification models loaded")

# ============================================================
# STEP 2: DEFINE PREPROCESSING FUNCTIONS (Same as Training)
# ============================================================

print("🔧 STEP 2: Setting up preprocessing pipeline...")

def roman_urdu_normalization(text):
    """Roman Urdu specific text normalization"""
    variations = {
        r'\bsee\b': 'si', r'\bhey\b': 'hai', r'\bhae\b': 'hai',
        r'\btum\b': 'tu', r'\bap\b': 'aap', r'\bk\b': 'ke',
        r'\bka\b': 'ka', r'\bki\b': 'ki', r'\bbs\b': 'bas',
        r'\bplz\b': 'please', r'\bu\b': 'you', r'\br\b': 'are'
    }

    for pattern, replacement in variations.items():
        text = re.sub(pattern, replacement, text)
    return text

def preprocess_comment(text):
    """Complete preprocessing pipeline (same as training)"""
    text = str(text).lower()

    # Remove URLs, mentions, hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Roman Urdu normalization
    text = roman_urdu_normalization(text)

    # Conservative repeated character normalization
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)

    # Selective punctuation handling
    text = re.sub(r'[#$%&*()+,\-/:;<=>@[\\]^_`{|}~]', ' ', text)

    # Preserve emotional punctuation
    text = re.sub(r'!+', ' ! ', text)
    text = re.sub(r'\?+', ' ? ', text)
    text = re.sub(r'\.{2,}', ' ... ', text)

    # Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def comment_to_vector(comment, fasttext_model):
    """Convert comment to FastText sentence vector"""
    words = str(comment).split()
    word_vectors = []

    for word in words:
        try:
            vector = fasttext_model.get_word_vector(word)
            word_vectors.append(vector)
        except:
            continue

    if len(word_vectors) == 0:
        return np.zeros(fasttext_model.get_dimension())

    return np.mean(word_vectors, axis=0)

print("✅ Preprocessing pipeline ready")

# ============================================================
# STEP 3: TEST WITH REAL COMMENTS
# ============================================================

print("\n🎯 STEP 3: Testing with real comments...")

# Sample comments for testing (mix of abusive and non-abusive)
test_comments = [
    # ABUSIVE COMMENTS
    "tu bahut harami insan hai",
    "kuttiya ki aulad",
    "teri maa ki gaali deni hai",
    "bhenchod harami",
    "laanat hai tum par",

    # NON-ABUSIVE COMMENTS
    "bahut acha video hai bhai",
    "main ne kal movie dekhi",
    "yeh to funny hai",
    "subscribe to my channel",
    "nice content bro"
]

print("📝 Test Comments:")
for i, comment in enumerate(test_comments, 1):
    print(f"  {i}. {comment}")

# ============================================================
# STEP 4: PREDICT WITH ALL MODELS
# ============================================================

print("\n🔮 STEP 4: Making predictions with all models...")
print("=" * 70)

for comment in test_comments:
    print(f"\n💬 Comment: '{comment}'")
    print("-" * 50)

    # 1. Preprocess the comment
    cleaned_comment = preprocess_comment(comment)
    print(f"🧹 Cleaned: '{cleaned_comment}'")

    # 2. Convert to FastText vector
    comment_vector = comment_to_vector(cleaned_comment, fasttext_model)
    comment_vector = comment_vector.reshape(1, -1)  # Reshape for prediction

    # 3. Get predictions from all models
    for model_name, model in models.items():
        try:
            # Predict
            prediction = model.predict(comment_vector)[0]

            # Get probability if available
            if hasattr(model, 'predict_proba'):
                probability = model.predict_proba(comment_vector)[0]
                confidence = max(probability)
                abusive_prob = probability[1] if len(probability) > 1 else probability[0]
            else:
                confidence = "N/A"
                abusive_prob = "N/A"

            # Convert back to original label
            predicted_label = label_encoder.inverse_transform([prediction])[0]

            print(f"  🤖 {model_name:20} → {predicted_label:10} "
                  f"(Confidence: {confidence:.3f}, Abusive Prob: {abusive_prob:.3f})")

        except Exception as e:
            print(f"  ❌ {model_name:20} → Error: {e}")

# ============================================================
# STEP 5: BATCH TESTING WITH DETAILED ANALYSIS
# ============================================================

print("\n📊 STEP 5: Batch testing with detailed analysis...")
print("=" * 70)

# Test with more diverse comments
extended_test_comments = [
    "kameena insan", "badtameez larki", "ullu ka patha",  # Abusive
    "wonderful video", "amazing content", "love it",       # Non-abusive
    "kya baat hai", "maza aa gya", "super bro",            # Positive Roman Urdu
    "chod de yaar", "beghairat admi", "kutte ki maut"      # Abusive Roman Urdu
]

print("🔍 Extended Test Results:")
print("-" * 70)

results = []
for comment in extended_test_comments:
    cleaned = preprocess_comment(comment)
    vector = comment_to_vector(cleaned, fasttext_model).reshape(1, -1)

    model_predictions = {}
    for model_name, model in models.items():
        try:
            pred = model.predict(vector)[0]
            label = label_encoder.inverse_transform([pred])[0]
            model_predictions[model_name] = label
        except:
            model_predictions[model_name] = "Error"

    results.append({
        'Original': comment,
        'Cleaned': cleaned,
        **model_predictions
    })

# Display results in a clean table
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# ============================================================
# STEP 6: MODEL AGREEMENT ANALYSIS
# ============================================================

print("\n🤝 STEP 6: Model Agreement Analysis...")
print("=" * 70)

agreement_count = 0
total_comments = len(extended_test_comments)

for result in results:
    predictions = [result['Logistic Regression'], result['SVM'], result['Random Forest']]
    if len(set(predictions)) == 1:  # All models agree
        agreement_count += 1

print(f"📈 Models agree on {agreement_count}/{total_comments} comments "
      f"({agreement_count/total_comments*100:.1f}% agreement)")

print("\n🎯 TESTING COMPLETED! All models are working with proper preprocessing!")

🧪 TESTING SAVED MODELS WITH REAL COMMENTS...
📥 STEP 1: Loading all saved models...
✅ FastText model loaded
✅ Label encoder loaded
✅ All classification models loaded
🔧 STEP 2: Setting up preprocessing pipeline...
✅ Preprocessing pipeline ready

🎯 STEP 3: Testing with real comments...
📝 Test Comments:
  1. tu bahut harami insan hai
  2. kuttiya ki aulad
  3. teri maa ki gaali deni hai
  4. bhenchod harami
  5. laanat hai tum par
  6. bahut acha video hai bhai
  7. main ne kal movie dekhi
  8. yeh to funny hai
  9. subscribe to my channel
  10. nice content bro

🔮 STEP 4: Making predictions with all models...

💬 Comment: 'tu bahut harami insan hai'
--------------------------------------------------
🧹 Cleaned: 'tu bahut harami insan hai'
  🤖 Logistic Regression  →          1 (Confidence: 0.958, Abusive Prob: 0.958)
  ❌ SVM                  → Error: Unknown format code 'f' for object of type 'str'
  🤖 Random Forest        →          1 (Confidence: 0.720, Abusive Prob: 0.720)

💬 Comment: 'ku